# Modelling Portfolio accumulation

* Modelling a passive investor who purchases `n_block_val` worth of shares every `block_period` (unit of time). 
  * `n_block_val` is inflation adjusted at a constant rate of 2.2% p.a.
  * `block_period = 4 weeks`, for simplicity.
* Output 1: arithmetic return after `hold_duration`. That is simply portfolio_value(T)/total_cost_of_purchases.

### Notes

* Any kind of unit returns p.a. (arithmetic, CAGR) are not a very meaningful measure, because you're gradually buying more of the same asset, not, say, buying the entire asset all at once and repaying with debt. Strictly speaking, the normalized way of assessing ETF return to other asset classes would then be NPV. 
* As such, we simply use asset value at the end of the period. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import math

from datetime import datetime, timedelta, date
import tqdm as tqdm


def get_cagr(s0, sT, hold_duration):
    """Calculate CAGR"""
    return ((sT/s0)**(1/hold_duration)) - 1


def get_nearest_date_row(df, target_date):
    """Get the row in df closest to the input target_date.
    Note that the approximation can only occur *later* than the target_date, 
    but not before.
    
    PARAMS
    ------
    df: input dataframe
    target_date: datetime.datetime object.
    
    RETURNS
    -------
    row_dict: dictionary of row values.
    """        
    date_window_ls = [target_date]
    window_size = 21
    for i in np.arange(1, window_size):
        date_window_ls.append(target_date+timedelta(days=int(i)))
    # Get all values within the target date window, but keep only the first (earliest) row
    row_T = df.loc[df["date_obj"].isin(date_window_ls)].iloc[0]
    row_dict = {}
    for idx, val in row_T.iteritems():
        row_dict[idx] = val
        
    return row_dict


In [2]:
d0 = pd.read_csv("data/^GSPC_daily.csv")
d0["date_obj"] = d0.apply(lambda row: datetime.fromisoformat(str(row["Date"])), axis=1)
# set earliest date, if you want
d0 = d0.loc[d0["date_obj"]>datetime.fromisoformat("1950-01-01")]

# ========== params ==========
total_duration = 365*10 # total holding duration, in days
purchase_budget = 5000 # max value of each purchase
purchase_budget_r = 0.02 # How much to increase purchase_block_val by per year. UNUSED. 
inflation_r = 0.02 # Inflation rate, assumed to be a constant term. UNUSED.
purchase_period = 120 # how long between purchases, in days
v0 = 0 # starting value
# ========== calc'd params ==========
final_date = datetime.fromisoformat(d0.iloc[-1].Date)

### Start Full Run

Compute the final 10-y portfolio values of every possible entry point from 1950-01-01, to 2009-01-01. 

In [ ]:
datex = datetime.fromisoformat("1970-01-01")
t0_vec = [datex]
date_diff = datetime.fromisoformat("2009-01-01") - datex
for i in np.arange(1, date_diff.days):
    t0_vec.append(datex+timedelta(days=int(i)))

meta_contents = []
for t0 in tqdm.tqdm(t0_vec):
    purchase_vec = []

    t_x = t0
    contents = []
    while t_x < t0 + timedelta(days=total_duration):
        t_x += timedelta(days=purchase_period)
        row_dict = get_nearest_date_row(d0, t_x)
        new_row = [row_dict[k] for k in list(row_dict.keys())]
        contents.append(new_row)
    colnames = list(row_dict.keys())
    dt = pd.DataFrame(data=contents, columns=colnames)

    dt["n_units_purchased"] = dt.apply(lambda row: math.floor(purchase_budget/float(row["Adj Close"])), axis=1)
    dt["total_purchase_val"] = dt.apply(lambda row: float(row["n_units_purchased"])*float(row["Adj Close"]), axis=1)
    
    total_units = np.sum(dt["n_units_purchased"])
    total_cost = np.sum(dt["total_purchase_val"])
    total_final_val = dt.iloc[-1]["Adj Close"] * total_units
    multiple = total_final_val/total_cost

    meta_contents.append([total_units, total_cost, total_final_val, multiple])
    
# put results in a dataframe
dz = pd.DataFrame(data=meta_contents, columsn=["total_units", "total_cost", "total_final_val", "multiple"])

 75%|███████▌  | 10745/14245 [15:08<07:30,  7.78it/s]

In [ ]:
plt.hist(list(dz["multiple"]), bins=80)
plt.show()

In [ ]:
len(dz.loc[dz["multiple"]<(1.025**10)])/len(dz)

In [ ]:
dz